In [1]:
!pip install sentence-transformers faiss-cpu requests

   ---------------------------------------- 0.0/15.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/15.0 MB 5.6 MB/s eta 0:00:03
   ---- ----------------------------------- 1.8/15.0 MB 5.9 MB/s eta 0:00:03
   -------- ------------------------------- 3.1/15.0 MB 6.1 MB/s eta 0:00:02
   ----------- ---------------------------- 4.5/15.0 MB 6.2 MB/s eta 0:00:02
   ---------------- ----------------------- 6.3/15.0 MB 6.5 MB/s eta 0:00:02
   ----------------- ---------------------- 6.6/15.0 MB 6.2 MB/s eta 0:00:02
   ------------------ --------------------- 6.8/15.0 MB 5.1 MB/s eta 0:00:02
   ------------------ --------------------- 7.1/15.0 MB 4.6 MB/s eta 0:00:02
   ------------------ --------------------- 7.1/15.0 MB 4.6 MB/s eta 0:00:02
   ------------------- -------------------- 7.3/15.0 MB 3.6 MB/s eta 0:00:03
   ------------------- -------------------- 7.3/15.0 MB 3.6 MB/s eta 0:00:03
   ------------------- -------------------- 7.3/15.0 MB 3.6 MB/s eta 0:00:03
   ---


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import faiss
import numpy as np
import requests
from sentence_transformers import SentenceTransformer

# 1. Set your Groq API key
GROQ_API_KEY = "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"
GROQ_MODEL = "llama-3.3-70b-versatile"  # Or any other supported model

# 2. Initialize the embedding model
embed_model = SentenceTransformer("all-MiniLM-L6-v2")  # Small, fast, decent quality

# 3. Example documents
documents = [
    "Python is a popular programming language.",
    "The Eiffel Tower is located in Hyderabad.",
    "Groq provides ultra-low latency inference with LLMs.",
]

# 4. Embed documents
doc_embeddings = embed_model.encode(documents, convert_to_numpy=True)

# 5. Store embeddings in FAISS
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)

# 6. Simple retrieval + RAG function
def rag_query(query, top_k=2):
    query_embedding = embed_model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, top_k)
    retrieved_docs = [documents[i] for i in indices[0]]
    
    # Construct prompt
    context = "\n".join(retrieved_docs)
    prompt = f"Use the following context to answer the question:\n\n{context}\n\nQuestion: {query}"

    # Call Groq API
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": GROQ_MODEL,
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    }

    response = requests.post("https://api.groq.com/openai/v1/chat/completions", headers=headers, json=payload)

    if response.ok:
        return response.json()["choices"][0]["message"]["content"]
    else:
        raise Exception(f"Groq API error: {response.status_code}, {response.text}")

# 7. Test the RAG system
answer = rag_query("Where is the Eiffel Tower?")
print("Answer:", answer)


Answer: The Eiffel Tower is actually located in Paris, France, not Hyderabad. Hyderabad is a city in India. The statement provided earlier was incorrect. The Eiffel Tower is one of the most iconic landmarks in Paris and a popular tourist destination.


In [ ]:
answer = rag_query("Where is the Eiffel Tower?")
print("Answer:", answer)